### 1.2.2 レイヤとしてのクラス化と順伝播の実装

- Python版ではレイヤをclassで定義していたが，Juliaではclassの概念が存在しない．
  - モデルをコンストラクタで定義し，それにメソッドを作用させていくように考える．
- そのため，レイヤに持たせていたメソッド`forward()`と`backward()`，インスタンス変数`params`と`grads`を独立させる．
  - メソッドは`function`で定義し，インスタンス変数は`struct`に持たせる．
  - Juliaの多重ディスパッチの機構を利用し，同じ名前のメソッドでも型によって異なる処理を呼び出す．
    - レイヤを型として定義する．

In [1]:
# 抽象型の定義
abstract type Layer end

In [2]:
# Sigmoidレイヤの定義
mutable struct Sigmoid <: Layer
    params
    # コンストラクタメソッド
    Sigmoid() = new()
end

In [3]:
# Affineレイヤの定義
mutable struct Affine <: Layer
    params
    W
    b
    function Affine(W, b)
        params = (W, b)
        new(params)
    end
end

In [4]:
# Python版のSigmoidクラスのforward()
function forward(self::Sigmoid, x) # デフォルトの引数を指定することを考慮して，selfを最初に指定する
    return 1.0 ./ (1.0 .+ exp.(-x))
end

forward (generic function with 1 method)

In [5]:
# Python版のAffineクラスのforward()
function forward(self::Affine, x)
    W, b = self.params
    return x * W .+ reshape(b, 1, :) # bを1x4 Arrayにreshape
end

forward (generic function with 2 methods)

In [6]:
# TwoLayerNetの定義
mutable struct TwoLayerNet
    I # input_size
    H # hidden_size
    O # output_size
    W1
    b1
    W2
    b2
    layers
    
    # コンストラクタメソッド
    function TwoLayerNet(I, H, O)
        # 重みとバイアスの初期化
        W1 = randn(I, H)
        b1 = randn(H)
        W2 = randn(H, O)
        b2 = randn(O)
        
        # レイヤの生成
        layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        new(I, H, O, W1, b1, W2, b2, layers)
    end
end

In [7]:
# Python版のTwoLayerNetのpredict()
function predict(self::TwoLayerNet, x)
    for layer in self.layers
        x = forward(layer, x)
    end
    return x
end

predict (generic function with 1 method)

In [8]:
x = randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = predict(model, x)

10×3 Array{Float64,2}:
 1.04541    0.626532   0.715811
 0.894227   0.93156    0.594773
 0.689666   0.97653    0.399264
 1.12305    0.731338   0.772146
 1.33759   -0.0305813  0.960755
 0.936619   1.05838    0.635785
 0.783455   0.706819   0.50651
 0.969139   0.607946   0.655325
 0.868811   0.747106   0.573951
 0.776047   1.02521    0.477398